Process the data...

In [1]:
data_dir = ".\\data\\"

#filename = "2018-06-20-10-44-11.tdms"
#keyence_filename= "2018-06-20-10-44-18-KEYENCE.tdms"

#filename = "2018-07-12-18-36-41.tdms"
#keyence_filename= "2018-07-12-18-36-56-KEYENCE.tdms"

#filename = "2018-07-12-18-51-08.tdms"
#keyence_filename= "2018-07-12-18-51-17-KEYENCE.tdms"

#filename = "2018-07-17-10-17-29.tdms"
filename = "2018-07-17-09-45-52.tdms"
keyence_filename = None

In [2]:
import pickle

if keyence_filename is not None:
    pickle_file = data_dir + "_".join([filename, keyence_filename, "full_data.p"])
else:
    pickle_file = data_dir + "_".join([filename, "full_data.p"])

try:
    full_data = pickle.load(open(pickle_file, 'rb'))
    print("Loaded the data from " + pickle_file)
except FileNotFoundError:
    print("File not found, load the data again!")
    
# Pop the info out, it can't be processed...
info = full_data.pop("info")

Loaded the data from .\data\2018-07-17-09-45-52.tdms_full_data.p


In [3]:
print("Full dataset:")
for k in full_data:
    print("{}: \t{} points".format(k, len(full_data[k][[*full_data[k]][0]])))

Full dataset:
set_point: 	599 points
position: 	37248 points
temperature: 	1877 points


In [4]:
from data_tools import *

#limits = [full_data['position']['time'][5000], full_data['position']['time'][5100]]
limits = [full_data['set_point']['time'][0], full_data['set_point']['time'][-1]]

print("Using limits:")
pprint(limits)

# Truncate the data
trunc_data = dict(zip(full_data.keys(), [truncate(limits, full_data[k]) for k in full_data]))

print("Truncated dataset:")
for k in trunc_data:
    print("{}: \t{} points".format(k, len(trunc_data[k][[*trunc_data[k]][0]])))

Using limits:
[datetime.datetime(2018, 7, 17, 8, 48, 24, 621007, tzinfo=<UTC>),
 datetime.datetime(2018, 7, 17, 9, 5, 5, 153543, tzinfo=<UTC>)]
Truncated dataset:
set_point: 	598 points
position: 	19947 points
temperature: 	1000 points


In [5]:
# Choose what to work with:
#data = trunc_data
data = full_data

print("Using dataset:")
for k in data:
    print("{}: \t{} points".format(k, len(data[k][[*data[k]][0]])))

Using dataset:
set_point: 	599 points
position: 	37248 points
temperature: 	1877 points


In [6]:
# Find the data rates
rate = dict()

for k in data:
    time_delta = [u - t for t, u in zip(data[k]['time'], data[k]['time'][1:])]
    rate[k] = np.mean(time_delta).total_seconds()

print("Data rates (s):")                  
pprint(rate)


Data rates (s):
{'position': 0.050124, 'set_point': 1.673131, 'temperature': 1.000059}


In [7]:
# Calculate transition time (only needs to be run once)

from ipywidgets import FloatProgress
from IPython.display import display

# Round the position to 3dp
data['position']['round'] = list(np.around(data['position']['data'], 2))

# Initialise the transition time
data['set_point']['trans'] = [None,] * len(data['set_point']['time'])

# Make a progress bar
print("Calculating transition time")
p = FloatProgress(min=0, max=len(data['set_point']['time']))
display(p)

i_last = 0

for i, (t, sp) in enumerate(zip(data['set_point']['time'], data['set_point']['data'])):
    # Find the nearest time index in the position dataset
    i_start = find_nearest_index(data['position']['time'][i_last:], t) + i_last
    
    try:
        # Find the first index where the setpoint is reached from the rounded position dataset
        i_stop  = data['position']['round'][i_start:].index(sp) + i_start
        i_last = i_stop
        
        # The transition time is the time between the setpoint change, and the setpoint being reached
        time = data['position']['time'][i_stop] - t
        data['set_point']['trans'][i] = time.total_seconds()
    
    except ValueError:
        data['set_point']['trans'][i] = None
    
    # Update the progress bar
    p.value += 1

Calculating transition time


FloatProgress(value=0.0, max=599.0)

In [8]:
# Pairs to find the difference of pair[0] - pair[1], and store it in pair[0]
# Probably only makes sense to do this for pair[0] faster than pair[1]???
diff_pairs = [
    (['position', 'data'],            ['set_point', 'data'])
    ]

if keyence_filename is not None:
    diff_pairs.extend([
    (['keyence', ['data1', 'data2']], ['set_point', 'data']),
    (['keyence', ['data1', 'data2']], ['position',  'data']),
    ])

# Make a progress bar
from ipywidgets import FloatProgress
from IPython.display import display
print("Calculating differences")
p = FloatProgress()
display(p)

calculate_differences(diff_pairs, data, progress=p)

Calculating differences


FloatProgress(value=0.0)

In [9]:
# Pairs to create a mask on pair[0] using value changes of pair[1] and a margin, and store it in pair[0]
# Probably only makes sense to do this for pair[0] faster than pair[1]???
mask_pairs = [('position', 'set_point')]
jitter = rate['position']

if keyence_filename is not None:
    mask.pairs.extend([('keyence',  'set_point')])
    jitter = max([rate['position'], rate['keyence']])


trans = np.mean(data['set_point']['trans'][:-1])

mask_margin = [datetime.timedelta(seconds =     0 - jitter),
               datetime.timedelta(seconds = trans + jitter)]

print("Mask margin \t{}s".format(mask_margin))

# Make a progress bar
from ipywidgets import FloatProgress
from IPython.display import display
print("Calculating masks")
p = FloatProgress()
display(p)

generate_mask(mask_pairs, mask_margin, data, progress=p)

Mask margin 	[datetime.timedelta(-1, 86399, 949876), datetime.timedelta(0, 1, 223612)]s
Calculating masks


FloatProgress(value=0.0)

In [10]:
for k in data:
    print(k)
    for l in data[k]:
        print("\t"+l)

set_point
	time
	data
	trans
position
	time
	data
	round
	data_set_point_data
	mask_set_point
temperature
	time
	stage
	base
	amb


In [11]:
# Store the processed data...

if keyence_filename is not None:
    pickle_file = data_dir + "_".join([filename, keyence_filename, "proc_data.p"])
else:
    pickle_file = data_dir + "_".join([filename, "proc_data.p"])

# Add the info back in before pickling...
data['info'] = info

pickle.dump(data, open(pickle_file, 'wb'))

print("Processed dataset stored in {}".format(pickle_file))

Processed dataset stored in .\data\2018-07-17-09-45-52.tdms_proc_data.p
